In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
from string import Template
import os
import re
from collections import defaultdict
import random
import json
%matplotlib inline


In [2]:
from datetime import datetime, timedelta

dateString = "2013/12/21"
dateFormatter = "%Y/%m/%d"
datetime.strptime(dateString, dateFormatter)
test_symbols = ['2330', '3008', '2317', '2412']
DEBUG = False

In [3]:
# Test broker id fetch 
def fetch_brokerId():
    global BkrMap
    URL = 'https://fubon-ebrokerdj.fbs.com.tw/z/js/zbrokerjs.djjs'
    resp = requests.get(URL)
    resp.encoding = 'cp950' # 中文
    #print (resp.text)
    reBkr = re.compile(r".*g_BrokerList = \'(.*)\'", re.MULTILINE|re.DOTALL)
    reBkr2Id = re.compile(r"(.*),(.*)", re.DOTALL)
    BkrGrps = reBkr.match(resp.text).group(1).split(';') # slice to broker groups
    BkrMap = defaultdict(lambda: defaultdict(str))

    for bkrgrp in BkrGrps:
        bkrlist = bkrgrp.split('!') # slice to sub broker
        for i, bkr in enumerate(bkrlist):
            bmt = reBkr2Id.match(bkr)
            if bmt:
                top_bkr = bmt.group(2) if i == 0 else top_bkr
                BkrMap[top_bkr].update({bmt.group(2): bmt.group(1)}) #group(1) = b

    # debug            
    #for k in sorted(BkrMap.keys())[:1]:
    #    print(BkrMap[k])
fetch_brokerId()

In [4]:
# random pick BHID, SBHIDS
def random_pick_BHIDs():
    global BkrMap
    test_BHIDS = list(BkrMap[random.choice(list(BkrMap.keys()))].items())
    print (test_BHIDS)
    BHID = test_BHIDS[0][1]
    SBHID = random.choice(test_BHIDS[1:])[1] if len(test_BHIDS) > 2 else BHID # if have sub SBHIDS
    #print (BHID, SBHID)
    return (BHID, SBHID)
    

In [5]:
BHID = '8150' # 台新
SBHID = '8151' # 台新建北

if not DEBUG:
    BHID, SBHID = random_pick_BHIDs()
    
print (BHID, SBHID)

[('台灣匯立證券', '1380')]
1380 1380


In [27]:
#http://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=2330&BHID=9800&b=9800
# get broker stock position detail put/call volume
def get_broker_sub_symbol_position(BHID, SBHID, symbol):
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=$SYMBOL&BHID=$BHID&b=$SBHID')
    URLPar = {
        'SYMBOL': symbol,
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table', {'class': 't01'})[1]
    values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
    ddates = list(map(lambda x: datetime.strptime(str(x.text), dateFormatter), table.find_all('td', {'class': 't4n0'})))
    #print (values)
    #print (len(values))
    calls = [values[i] for i in range(0, len(values), 4)]  #買
    puts = [values[i+1] for i in range(0, len(values), 4)] #賣
    BkrPutCall = {
        'date': ddates,
        'symbol': symbol,
        'puts': puts,
        'calls': calls
    }
    return  pd.DataFrame(BkrPutCall)

In [28]:
symbol = '2330'
if not DEBUG:
    symbol = random.choice(test_symbols)

BkrPutCallpf = get_broker_sub_symbol_position(BHID, SBHID, symbol)
BkrPutCallpf[:4]

,date,symbol,puts,calls
0,2021-02-04,3008,1,0
1,2021-02-03,3008,0,246
2,2021-02-02,3008,14,73
3,2021-02-01,3008,4,0


In [29]:
#https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=1440&b=1440&c=E&d=1
def get_broker_all_symbol_position(BHID, SBHID):
    
    # parser GenLink2stk('AS3481','群創');
    reSymbol2Id = re.compile(r".*GenLink2stk\(\'AS(.*)\',\'(.*)\'\);", re.M|re.DOTALL)  
    
    #parser <a href="javascript:Link2Stk('00677U');">00677U富邦VIX</a>
    # no support yet
    #reSymbol2Id2 = re.compile(r".*Link2Stk\(\'(.*)\'\);\"\>(.*)\<\/a\>", re.M|re.DOTALL)
    
    def get_symbol(text): 
        smt = reSymbol2Id.match(text)
        if smt:
            return smt.group(1)
            
    def get_puts_calls(table, Bkrlist):
        values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
        symbols = list(map(lambda x:  get_symbol(str(x.text)), table.find_all('td', {'class': 't4t1'})))
        
        for i, symbol in enumerate(symbols):
            if symbol == None:
                continue
            Bkrlist.append({
            'symbol': symbol,
            'calls': values[i*3+0], #買
            'puts': values[i*3+1]   #賣
            })
        
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=$BHID&b=$SBHID&c=E&d=1')
    URLPar = {
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    tables = soup.find_all('table', {'class': 't0'})
    Bkrlist = list()
    get_puts_calls(tables[0], Bkrlist) # 買超
    get_puts_calls(tables[1], Bkrlist) # 賣超
    
    return pd.DataFrame(Bkrlist)

In [30]:
BHID2Symbolpf = get_broker_all_symbol_position(BHID, SBHID)
BHID2Symbolpf.head()

,symbol,calls,puts
0,2885,2858,119
1,2324,996,88
2,3576,609,0
3,2317,634,166
4,3081,451,0


In [10]:
from mongoengine import *

In [11]:
disconnect()
connect(alias='symbol-db-alias', db='symbol-db')
connect(alias='broker-db-alias', db='broker-db')
connect(alias='symbol-broker-db-alias', db='symbol-broker-db')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [12]:
#TopBroker = {
#  'BrokerId' : {
#  [
#       'date0': { [{'symbolId', 'puts', 'calls'}, ...]},
#]
#}
#}
class BrokerPositionDB(EmbeddedDocument): #embed
    symbol = IntField() # ref symbol to Symbol Top 
    puts = IntField()   # puts volume
    calls = IntField()  # call volume

class BrokerDB(Document):
    date = DateTimeField(default=datetime.utcnow)
    position = ListField(EmbeddedDocumentField(BrokerPositionDB))
    
    meta = {'db_alias': 'broker-db-alias'}
    
class BarDB(Document):
    O = FloatField()
    L = FloatField()
    V = IntField()
    
#MarginPS(Document):

class SymbolDB(Document):
    date = DateTimeField(default=datetime.utcnow)
    broker = ReferenceField(BrokerDB)
    bar = ReferenceField(BarDB)

    meta = {'db_alias': 'symbol-broker-db-alias'}
    

In [13]:
#bkrPost = BkrPutCallpf.iloc[0].to_dict()
BrkSymbolDic = BHID2Symbolpf.to_dict(orient="records")

In [31]:
# clean BrokerDB as empty
BrokerDB.drop_collection()

# random gen 10 broker symbols to mongodb
for i in range(10):
    brokerDb = BrokerDB()
    brokerDb.date = datetime.now() - timedelta(days=i)
    # pipeline from scrapy to MongoeEngine store
    brokerDb.position = [BrokerPositionDB(**bb) for bb in BrkSymbolDic]
    brokerDb.save()

#broker.objects

In [85]:
obj = BrokerDB.objects()[0]
print(BrokerDB.objects.count())

# test find max count of call symbols at sample time window
pipeline = [
    {"$match": { "date": { "$lte": datetime.now()} } },
    {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2) } } },
    {"$unwind": "$position" },
    {"$group": { "_id": "$position.symbol", "sum_puts": { "$sum": "$position.puts" }, "sum_calls": { "$sum": "$position.calls" } }  },
    {"$sort": { "sum_calls": -1, "sum_puts": 1 } },
    {"$limit": 3 }
    ]
collect = BrokerDB.objects().aggregate(pipeline)
items = [cursor for cursor in list(collect)] # clean cursor ptr and store to list
for it in items:
    print (it)
    
MaxCallSymbol = items[0]['_id']
MaxCallSymbol

10
{'_id': 3037, 'sum_puts': 16, 'sum_calls': 6014}
{'_id': 1101, 'sum_puts': 48, 'sum_calls': 5954}
{'_id': 3105, 'sum_puts': 2, 'sum_calls': 3566}


3037

In [62]:
# test find symbol's call event at sample time window
pipeline = [
    {"$match": { "date": { "$lte": datetime.now()} } },
    {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2) } } },
    {"$unwind": "$position" },
    {"$match": { "position.symbol": MaxCallSymbol } },
    {"$sort": { "date": 1} }
    ]
collect = BrokerDB.objects().aggregate(pipeline)
items = [cursor for cursor in list(collect)] # clean cursor ptr and store to list
# put to abu backtest algorithm to test broker win rate
for it in items:
    print (it['date'])

2021-02-03 20:35:32.593000
2021-02-04 20:35:32.570000


In [91]:
# test find max time of call(at least Call 5units threshold) symbols at sample time window
# stage1
pipeline = [
    {"$match": { "date": { "$lte": datetime.now()} } },
    {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2) } } },
    {"$match": { "position.calls": { "$gte": 5} } },
    {"$match": { "position.puts": { "$lte": 5} } },
    {"$unwind": "$position" },
    {"$group": { "_id": "$position.symbol", "times": { "$sum": 1 } } },
    {"$sort":  { "times": -1 } },
    {"$limit": 3 }
    ]
collect = BrokerDB.objects().aggregate(pipeline)
items = [cursor for cursor in list(collect)] # clean cursor ptr and store to list
for it in items:
    print (it)
    
MaxCallSymbol = items[0]['_id']
MaxCallSymbol


{'_id': 2330, 'times': 2}
{'_id': 2313, 'times': 2}
{'_id': 3708, 'times': 2}


2330

In [93]:
# test find max time of Daily punch(隔(n)日沖) symbols at sample time window
# stage0
pipeline = [
    {"$match": { "date": { "$lte": datetime.now()} } },
    {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2) } } },
    {"$match": { "position.calls": { "$gte": 5} } },
    {"$match": { "position.puts": { "$gte": 5} } }
    ]
collect = BrokerDB.objects().aggregate(pipeline)
items = [cursor for cursor in list(collect)] # clean cursor ptr and store to list
for it in items:
    print (it)
    
MaxCallSymbol = items[0]['_id']
MaxCallSymbol



{'_id': ObjectId('601bea149e75e0420e607eb2'), 'date': datetime.datetime(2021, 2, 4, 20, 35, 32, 570000), 'position': [{'symbol': 2885, 'puts': 2858, 'calls': 119}, {'symbol': 2324, 'puts': 996, 'calls': 88}, {'symbol': 3576, 'puts': 609, 'calls': 0}, {'symbol': 2317, 'puts': 634, 'calls': 166}, {'symbol': 3081, 'puts': 451, 'calls': 0}, {'symbol': 3481, 'puts': 400, 'calls': 0}, {'symbol': 2308, 'puts': 442, 'calls': 141}, {'symbol': 6239, 'puts': 286, 'calls': 0}, {'symbol': 2633, 'puts': 189, 'calls': 0}, {'symbol': 2382, 'puts': 193, 'calls': 11}, {'symbol': 2393, 'puts': 175, 'calls': 0}, {'symbol': 6669, 'puts': 137, 'calls': 0}, {'symbol': 6712, 'puts': 100, 'calls': 0}, {'symbol': 2883, 'puts': 93, 'calls': 0}, {'symbol': 5347, 'puts': 102, 'calls': 22}, {'symbol': 2887, 'puts': 67, 'calls': 0}, {'symbol': 2891, 'puts': 64, 'calls': 3}, {'symbol': 2605, 'puts': 61, 'calls': 0}, {'symbol': 2105, 'puts': 56, 'calls': 0}, {'symbol': 9911, 'puts': 56, 'calls': 0}, {'symbol': 2206, '

ObjectId('601bea149e75e0420e607eb2')